In [1]:
import os

In [2]:
%pwd

'd:\\Nikku\\End to End Project\\Wine-Quality-Tester\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Nikku\\End to End Project\\Wine-Quality-Tester'

In [5]:
import pandas as pd

In [7]:
data =pd.read_csv("artifacts\data_ingestion\winequality-red.csv")
data.sample(4)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
785,9.9,0.35,0.41,2.3,0.083,11.0,61.0,0.99820,3.21,0.50,9.5,5
427,9.5,0.78,0.22,1.9,0.077,6.0,32.0,0.99880,3.26,0.56,10.6,6
1319,9.1,0.76,0.68,1.7,0.414,18.0,64.0,0.99652,2.90,1.33,9.1,6
1238,8.1,0.73,0.00,2.5,0.081,12.0,24.0,0.99798,3.38,0.46,9.6,4


In [8]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Columns: 12 entries, fixed acidity to quality
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [10]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [21]:
data.shape

(1599, 12)

In [27]:
data.dtypes[11]

dtype('int64')

In [22]:
data.columns.dtype()

TypeError: 'numpy.dtype[object_]' object is not callable

In [12]:
# Prepare the entity

from dataclasses import dataclass
from pathlib import  Path

@dataclass(frozen= True)
class DataValidationConfig:
    root_dir : Path
    STATUS_FILE : str
    unzip_data_dir : Path
    all_schema: dict

In [14]:
# Prepare the Configuration Manager

from winetester.constant import *
from winetester.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_validation_config(self) -> DataValidationConfig:
        config  = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir= config.unzip_data_dir,
            all_schema = schema,
            
        )
        return data_validation_config
        

In [34]:
# Update Compoents

import os
from winetester import logger



class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            
            all_schema = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"validation_status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"validation_status: {validation_status}")
            return validation_status
        except Exception as e:
            raise e
    
    # def validate_datatype(self) -> bool:
        
    #     try:
    #         validate_data = None
    #         data = pd.read_csv(self.config.unzip_data_dir)
    #         datatype = data.dtypes
            
    #         all_datatype = self.config.all_schema.values()
            
    #         for data in datatype:
    #             if data not in all_datatype:
    #                 validate_data = False
    #                 with open(self.config.STATUS_FILE,'w') as f:
    #                     f.write(f"validate_datatype: {validate_data}")
    #             else:
    #                 validate_data = True
    #                 with open(self.config.STATUS_FILE,'w') as f:
    #                     f.write(f"validate_datatype: {validate_data}")
                        
    #         return validate_data
    #     except Exception as e:
    #         raise e
        
        
            
            

In [38]:
# Create Pipeline

try:
    config = ConfigurationManager()
    data_validation_config = config.get_validation_config()
    data_validation =  DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
    # data_validation.validate_datatype()
except Exception as e:
    raise e

[2023-10-09 16:24:54,194: INFO: Yaml file  : config\config.yaml loadded successfully]
[2023-10-09 16:24:54,195: INFO: Yaml file  : params.yaml loadded successfully]
[2023-10-09 16:24:54,196: INFO: Yaml file  : schema.yaml loadded successfully]
[2023-10-09 16:24:54,197: INFO: created directory at : artifacts]
[2023-10-09 16:24:54,197: INFO: created directory at : artifacts/data_validation]
